# Residual Networks

Goal is to build very deep convolutional networks, using Residual Networks (ResNets). In theory, very deep networks can represent very complex functions; but in practice, they are hard to train. Residual Networks, introduced by [He et al.](https://arxiv.org/pdf/1512.03385.pdf), allow you to train much deeper networks than were previously practically feasible.

**In this assignment, I am going to :**
- Implement the basic building blocks of ResNets. 
- Put together these building blocks to implement and train a state-of-the-art neural network for image classification. 

In [0]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

## 1 - The problem of very deep neural networks

In recent years, neural networks have become deeper, with state-of-the-art networks going from just a few layers (e.g., AlexNet) to over a hundred layers.

* The main benefit of a very deep network is that it can represent very complex functions. It can also learn features at many different levels of abstraction, from edges (at the shallower layers, closer to the input) to very complex features (at the deeper layers, closer to the output). 
* However, using a deeper network doesn't always help. A huge barrier to training them is vanishing gradients: very deep networks often have a gradient signal that goes to zero quickly, thus making gradient descent prohibitively slow. 
* More specifically, during gradient descent, as you backprop from the final layer back to the first layer, you are multiplying by the weight matrix on each step, and thus the gradient can decrease exponentially quickly to zero (or, in rare cases, grow exponentially quickly and "explode" to take very large values). 
* During training, you might therefore see the magnitude (or norm) of the gradient for the shallower layers decrease to zero very rapidly as training proceeds:

Please see the [image](https://) to understand how gradient decrease rapidly for shallow layers.

Let's solve this problem by using Residual networks.

## 2- Building a Residual network

In ResNets, a "shortcut" or a "skip connection" allows the model to skip layers:

Please refer to the [image](https://)

The image on the left shows the "main path" through the network. The image on the right adds a shortcut to the main path. By stacking these ResNet blocks on top of each other, you can form a very deep network.

ResNet blocks with the shortcut makes it very easy for one of the blocks to learn an identity function. This means that we can stack on additional ResNet blocks with little risk of harming training set performance.

Two main types of blocks are used in a ResNet, depending mainly on whether the input/output dimensions are same or different. I am going to implement both of them: the "identity block" and the "convolutional block."

### 2.1 Identity block

The identity block is the standard block used in ResNets, and corresponds to the case where the input activation (say  a[l]) has the same dimension as the output activation (say  a[l+2]). To flesh out the different steps of what happens in a ResNet's identity block, [here](https://) is an alternative diagram showing the individual steps:

The upper path in image is the "shortcut path." The lower path is the "main path."

Lets implement a slightly more powerful version of this identity block, in which the skip connection "skips over" 3 hidden layers. It looks like [this](https://)

Here are individual steps.

First component of main path:

* The first CONV2D has F1 filters of shape (1,1) and a stride of (1,1). Its padding is "valid" and lets name it as conv_name_base + '2a'. Use 0 as the seed for the random initialization.
* The first BatchNorm is normalizing the 'channels' axis. Its name should be bn_name_base + '2a'.
* Then apply the ReLU activation function. This has no name and no hyperparameters.

Second component of main path:

* The second CONV2D has F2 filters of shape (f,f) and a stride of (1,1). Its padding is "same" and its name should be conv_name_base + '2b'. Use 0 as the seed for the random initialization.
* The second BatchNorm is normalizing the 'channels' axis. Its name should be bn_name_base + '2b'.
* Then apply the ReLU activation function. This has no name and no hyperparameters.

Third component of main path:

* The third CONV2D has F3 filters of shape (1,1) and a stride of (1,1). Its padding is "valid" and its name should be conv_name_base + '2c'. Use 0 as the seed for the random initialization.
* The third BatchNorm is normalizing the 'channels' axis. Its name should be bn_name_base + '2c'.
* Note that there is no ReLU activation function in this component.

Final step:

* The X_shortcut and the output from the 3rd layer X are added together.
* Then apply the ReLU activation function. This has no name and no hyperparameters.

In [0]:
def identity_block(X, f, filters, stage, block):
  """
  Implementation of the identity block as defined in Figure 4
    
  Arguments:
  X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
  f -- integer, specifying the shape of the middle CONV's window for the main path
  filters -- python list of integers, defining the number of filters in the CONV layers of the main path
  stage -- integer, used to name the layers, depending on their position in the network
  block -- string/character, used to name the layers, depending on their position in the network
  
  Returns:
  X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
  """
  
  # defining name basis
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  
  # Retrieve Filters
  F1, F2, F3 = filters
  
  # Save the input value. Will need this later to add back to the main path. 
  X_shortcut = X
  
  # First component of main path
  X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = Activation('relu')(X)
  
  # Second component of main path
  X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = Activation('relu')(X)
  
  # Third component of main path
  X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
  
  # Final step: Add shortcut value to main path, and pass it through a RELU activation
  X = Add()([X_shortcut,X])
  X = Activation('relu')(X)
  
  return X

### 2.2 Convolutional block

The ResNet "convolutional block" is the second block type. We use this type of block when the input and output dimensions don't match up. The difference with the identity block is that there is a CONV2D layer in the shortcut path. See [here](https://)

* The CONV2D layer in the shortcut path is used to resize the input X to a different dimension, so that the dimensions match up in the final addition needed to add the shortcut value back to the main path.
* For example, to reduce the activation dimensions's height and width by a factor of 2, we can use a 1x1 convolution with a stride of 2.
* The CONV2D layer on the shortcut path does not use any non-linear activation function. Its main role is to just apply a (learned) linear function that reduces the dimension of the input, so that the dimensions match up for the later addition step.

Details of convolutions are as follows:

First component of main path:

* The first CONV2D has F1 filters of shape (1,1) and a stride of (s,s). Its padding is "valid" and its name should be conv_name_base + '2a'. Use 0 as the glorot_uniform seed.
* The first BatchNorm is normalizing the 'channels' axis. Its name should be bn_name_base + '2a'.
* Then apply the ReLU activation function. This has no name and no hyperparameters.

Second component of main path:

* The second CONV2D has F2 filters of shape (f,f) and a stride of (1,1). Its padding is "same" and it's name should be conv_name_base + '2b'. Use 0 as the glorot_uniform seed.
* The second BatchNorm is normalizing the 'channels' axis. Its name should be bn_name_base + '2b'.
* Then apply the ReLU activation function. This has no name and no hyperparameters.

Third component of main path:

* The third CONV2D has F3 filters of shape (1,1) and a stride of (1,1). Its padding is "valid" and it's name should be conv_name_base + '2c'. Use 0 as the glorot_uniform seed.
* The third BatchNorm is normalizing the 'channels' axis. Its name should be bn_name_base + '2c'. Note that there is no ReLU activation function in this component.

Shortcut path:

* The CONV2D has F3 filters of shape (1,1) and a stride of (s,s). Its padding is "valid" and its name should be conv_name_base + '1'. Use 0 as the glorot_uniform seed.
* The BatchNorm is normalizing the 'channels' axis. Its name should be bn_name_base + '1'.

Final step:

* The shortcut and the main path values are added together.
* Then apply the ReLU activation function. This has no name and no hyperparameters.

In [0]:
def convolutional_block(X, f, filters, stage, block, s = 2):
  """
  Implementation of the convolutional block as defined in Figure 4
    
  Arguments:
  X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
  f -- integer, specifying the shape of the middle CONV's window for the main path
  filters -- python list of integers, defining the number of filters in the CONV layers of the main path
  stage -- integer, used to name the layers, depending on their position in the network
  block -- string/character, used to name the layers, depending on their position in the network
  s -- Integer, specifying the stride to be used
    
  Returns:
  X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
  """
    
  # defining name basis
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
    
  # Retrieve Filters
  F1, F2, F3 = filters
    
  # Save the input value
  X_shortcut = X


  ##### MAIN PATH #####
  # First component of main path 
  X = Conv2D(F1, (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = Activation('relu')(X)

  # Second component of main path
  X = Conv2D(F2, (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = Activation('relu')(X)

  # Third component of main path
  X = Conv2D(F3, (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

  ##### SHORTCUT PATH ####
  X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X)
  X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X)

  # Final step: Add shortcut value to main path, and pass it through a RELU activation
  X = Add()([X_shortcut,X])
  X = Activation('relu')(X)

  return X

## 3 - Building first ResNet model (50 layers)

We have the necessary blocks to build a very deep ResNet. [figure](https://) describes in detail the architecture of this neural network. "ID BLOCK" in the diagram stands for "Identity block," and "ID BLOCK x3" means we should stack 3 identity blocks together.

The details of this ResNet-50 model are:

* Zero-padding pads the input with a pad of (3,3)
* Stage 1:
 * The 2D Convolution has 64 filters of shape (7,7) and uses a stride of (2,2). Its name is "conv1".
 * BatchNorm is applied to the 'channels' axis of the input.
 * MaxPooling uses a (3,3) window and a (2,2) stride.
* Stage 2:
 * The convolutional block uses three sets of filters of size [64,64,256], "f" is 3, "s" is 1 and the block is "a".
 * The 2 identity blocks use three sets of filters of size [64,64,256], "f" is 3 and the blocks are "b" and "c".
* Stage 3:
 * The convolutional block uses three sets of filters of size [128,128,512], "f" is 3, "s" is 2 and the block is "a".
 * The 3 identity blocks use three sets of filters of size [128,128,512], "f" is 3 and the blocks are "b", "c" and "d".
* Stage 4:
 * The convolutional block uses three sets of filters of size [256, 256, 1024], "f" is 3, "s" is 2 and the block is "a".
 * The 5 identity blocks use three sets of filters of size [256, 256, 1024], "f" is 3 and the blocks are "b", "c", "d", "e" and "f".
* Stage 5:
 * The convolutional block uses three sets of filters of size [512, 512, 2048], "f" is 3, "s" is 2 and the block is "a".
 * The 2 identity blocks use three sets of filters of size [512, 512, 2048], "f" is 3 and the blocks are "b" and "c".
* The 2D Average Pooling uses a window of shape (2,2) and its name is "avg_pool".
* The 'flatten' layer doesn't have any hyperparameters or name.
* The Fully Connected (Dense) layer reduces its input to the number of classes using a softmax activation. Its name should be 'fc' + str(classes).



In [0]:
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape = (64, 64, 3), classes = 6):
  """
  Implementation of the popular ResNet50 the following architecture:
  CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
  -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

  Arguments:
  input_shape -- shape of the images of the dataset
  classes -- integer, number of classes

  Returns:
  model -- a Model() instance in Keras
  """
    
  # Define the input as a tensor with shape input_shape
  X_input = Input(input_shape)

    
  # Zero-Padding
  X = ZeroPadding2D((3, 3))(X_input)
    
  # Stage 1
  X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((3, 3), strides=(2, 2))(X)

  # Stage 2
  X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
  X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
  X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

  ### START CODE HERE ###

  # Stage 3 (≈4 lines)
  X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
  X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
  X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
  X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

  # Stage 4 (≈6 lines)
  X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

  # Stage 5 (≈3 lines)
  X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
  X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
  X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
  X = AveragePooling2D(pool_size=(2, 2))(X)
    
  ### END CODE HERE ###

  # output layer
  X = Flatten()(X)
  X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
  # Create model
  model = Model(inputs = X_input, outputs = X, name='ResNet50')

  return model

Now lets build model graph.

In [0]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)  

# Here classes = 6 because dataset I will be passing through this ResNet has 6 classes and model need to classify
# image in to one of these 6 classes.

Prior training a model, we need to configure the learning process by compiling the model.

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model is ready to be trained. Lets load the dataset.

We will be using SIGNS dataset. SIGNS dataset is a collection of 6 hand signs representing numbers from 0 to 5.

Goal is to classify image in to 6 catgories.

Sample of how data looks like in [here](https://)

In [18]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


Run the following cell to train the model on 20 epochs with a batch size of 32.

In [19]:
model.fit(X_train, Y_train, epochs = 20, batch_size = 32)

Epoch 1/20
1080/1080 [==============================] - 25s 23ms/step - loss: 2.9296 - acc: 0.2796
Epoch 2/20
1080/1080 [==============================] - 9s 8ms/step - loss: 1.7834 - acc: 0.4889
Epoch 3/20
1080/1080 [==============================] - 9s 8ms/step - loss: 1.2903 - acc: 0.5769
Epoch 4/20
1080/1080 [==============================] - 8s 8ms/step - loss: 1.2597 - acc: 0.6769
Epoch 5/20
1080/1080 [==============================] - 8s 8ms/step - loss: 1.0505 - acc: 0.7694
Epoch 6/20
1080/1080 [==============================] - 8s 8ms/step - loss: 1.5049 - acc: 0.6065
Epoch 7/20
1080/1080 [==============================] - 8s 7ms/step - loss: 1.3280 - acc: 0.6657
Epoch 8/20
1080/1080 [==============================] - 8s 8ms/step - loss: 1.3090 - acc: 0.6861
Epoch 9/20
1080/1080 [==============================] - 8s 8ms/step - loss: 1.6119 - acc: 0.5463
Epoch 10/20
1080/1080 [==============================] - 8s 7ms/step - loss: 1.2564 - acc: 0.7083
Epoch 11/20
1080/1080 [====

Let's see how this model (trained on 20 epochs) performs on the test set.

In [20]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

120/120 [==============================] - 3s 21ms/step
Loss = 0.9885895808537801
Test Accuracy = 0.8583333333333333


ResNet50 is a powerful model for image classification when it is trained for an adequate number of iterations and with large dataset.

#### Important points to remember.

* Very deep "plain" networks don't work in practice because they are hard to train due to vanishing gradients.
* The skip-connections help to address the Vanishing Gradient problem. They also make it easy for a ResNet block to learn an identity function.
* There are two main types of blocks: The identity block and the convolutional block.
* Very deep Residual Networks are built by stacking these blocks together.